In [1]:
import pandas as pd 

In [30]:
df = pd.read_csv("dataset-3.csv")

In [31]:
df.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


# Q1.Distance Matrix Calculation

In [32]:
# Create an empty DataFrame with unique IDs
unique_ids = sorted(set(df['id_start'].unique()) | set(df['id_end'].unique()))
distance_matrix = pd.DataFrame(index=unique_ids, columns=unique_ids)
distance_matrix.fillna(0, inplace=True)  # Fill with zeros initially

# Populate the distance matrix with cumulative distances
for _, row in df.iterrows():
    id_start, id_end, distance = row['id_start'], row['id_end'], row['distance']
    distance_matrix.at[id_start, id_end] += distance
    distance_matrix.at[id_end, id_start] += distance  # Accounting for bidirectional distances

# Print the resulting distance matrix
print(distance_matrix)

         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7      0.0      0.0      0.0      0.0      0.0   
1001402      9.7      0.0     20.2      0.0      0.0      0.0      0.0   
1001404      0.0     20.2      0.0     16.0      0.0      0.0      0.0   
1001406      0.0      0.0     16.0      0.0     21.7      0.0      0.0   
1001408      0.0      0.0      0.0     21.7      0.0     11.1      0.0   
1001410      0.0      0.0      0.0      0.0     11.1      0.0     15.6   
1001412      0.0      0.0      0.0      0.0      0.0     15.6      0.0   
1001414      0.0      0.0      0.0      0.0      0.0      0.0     18.2   
1001416      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001418      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001420      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001422      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001424      0.0      0.0      0.0    

# Q2.Unroll Distance Matrix

In [33]:


data_list = []

# Iterate through the distance matrix to populate the list
for id_start in distance_matrix.index:
    for id_end in distance_matrix.columns:
        if id_start != id_end:
            distance = distance_matrix.at[id_start, id_end]
            data_list.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})

# Create the unrolled DataFrame using concat
unrolled_df = pd.concat([pd.DataFrame(data_list)])

# Print the resulting unrolled DataFrame
print(unrolled_df)

      id_start   id_end  distance
0      1001400  1001402       9.7
1      1001400  1001404       0.0
2      1001400  1001406       0.0
3      1001400  1001408       0.0
4      1001400  1001410       0.0
...        ...      ...       ...
1801   1004356  1001470       0.0
1802   1004356  1001472       0.0
1803   1004356  1001488       4.0
1804   1004356  1004354       2.0
1805   1004356  1004355       0.0

[1806 rows x 3 columns]


# Q3.Finding IDs within Percentage Threshold

In [ ]:
# same output with diffrent logics
# Logic 1 for Q3

In [42]:
# Choose a reference value
reference_value = 1004356

# Filter DataFrame based on reference_value
reference_df = unrolled_df[unrolled_df['id_start'] == reference_value]

# Calculate the average distance for the reference value
reference_avg_distance = reference_df['distance'].mean()

# Calculate the 10% threshold range
lower_threshold = reference_avg_distance * 0.9
upper_threshold = reference_avg_distance * 1.1

# Filter IDs within the 10% threshold
filtered_ids = unrolled_df[(unrolled_df['distance'] >= lower_threshold) & (unrolled_df['distance'] <= upper_threshold)]['id_start']

# Remove duplicates and sort the list
sorted_filtered_ids = sorted(set(filtered_ids))

# Print the result
print(sorted_filtered_ids)

[]


In [44]:

# logic 2 for Q3
def find_ids_within_ten_percentage_threshold(df, reference_id):
    """
    Find all IDs whose average distance lies within 10% of the average distance of the reference ID.

    Args:
        df (pandas.DataFrame): DataFrame with columns 'id_start', 'id_end', and 'distance'.
        reference_id (int): Reference ID for which to calculate the average distance.

    Returns:
        pandas.DataFrame: DataFrame with IDs whose average distance is within the specified percentage threshold
                          of the reference ID's average distance.
    """
    # Filter DataFrame based on reference_id
    reference_df = df[df['id_start'] == reference_id]

    # Calculate the average distance for the reference ID
    reference_avg_distance = reference_df['distance'].mean()

    # Calculate the 10% threshold range
    lower_threshold = reference_avg_distance * 0.9
    upper_threshold = reference_avg_distance * 1.1

    # Filter IDs within the 10% threshold
    filtered_df = df[(df['distance'] >= lower_threshold) & (df['distance'] <= upper_threshold)]

    return filtered_df

# Example usage:
# Assuming unrolled_df is the DataFrame obtained from Question 2
reference_id_example = 1001400
result_df = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id_example)
print(result_df)


Empty DataFrame
Columns: [id_start, id_end, distance]
Index: []


# Q4.Calculate Toll Rate

In [45]:
import pandas as pd

def calculate_toll_rate(df):
    """
    Calculate toll rates for each vehicle type based on the unrolled DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame with columns 'id_start', 'id_end', 'distance', and 'vehicle_type'.

    Returns:
        pandas.DataFrame: DataFrame with toll rates calculated for each vehicle type.
    """
    # Define rate coefficients for each vehicle type
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    # Calculate toll rates for each vehicle type
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        df[vehicle_type] = df['distance'] * rate_coefficient

    return df

# Example usage:
# Assuming unrolled_df is the DataFrame obtained from Question 2
result_with_toll_rates = calculate_toll_rate(unrolled_df)
print(result_with_toll_rates)


      id_start   id_end  distance  moto    car     rv    bus  truck
0      1001400  1001402       9.7  7.76  11.64  14.55  21.34  34.92
1      1001400  1001404       0.0  0.00   0.00   0.00   0.00   0.00
2      1001400  1001406       0.0  0.00   0.00   0.00   0.00   0.00
3      1001400  1001408       0.0  0.00   0.00   0.00   0.00   0.00
4      1001400  1001410       0.0  0.00   0.00   0.00   0.00   0.00
...        ...      ...       ...   ...    ...    ...    ...    ...
1801   1004356  1001470       0.0  0.00   0.00   0.00   0.00   0.00
1802   1004356  1001472       0.0  0.00   0.00   0.00   0.00   0.00
1803   1004356  1001488       4.0  3.20   4.80   6.00   8.80  14.40
1804   1004356  1004354       2.0  1.60   2.40   3.00   4.40   7.20
1805   1004356  1004355       0.0  0.00   0.00   0.00   0.00   0.00

[1806 rows x 8 columns]


# Q5. Calculate Time-Based Toll Rates

In [46]:
result_with_toll_rates


,id_start,id_end,distance,moto,car,rv,bus,truck
0,1001400,1001402,9.7,7.76,11.64,14.55,21.34,34.92
1,1001400,1001404,0.0,0.00,0.00,0.00,0.00,0.00
2,1001400,1001406,0.0,0.00,0.00,0.00,0.00,0.00
3,1001400,1001408,0.0,0.00,0.00,0.00,0.00,0.00
4,1001400,1001410,0.0,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...
1801,1004356,1001470,0.0,0.00,0.00,0.00,0.00,0.00
1802,1004356,1001472,0.0,0.00,0.00,0.00,0.00,0.00
1803,1004356,1001488,4.0,3.20,4.80,6.00,8.80,14.40
1804,1004356,1004354,2.0,1.60,2.40,3.00,4.40,7.20


In [52]:
import pandas as pd
from datetime import time

# Assuming result_with_toll_rates is your DataFrame
# For example, result_with_toll_rates = pd.read_csv('your_data.csv')

# Define discount factors
weekday_discounts = {
    (time(0, 0, 0), time(10, 0, 0)): 0.8,
    (time(10, 0, 0), time(18, 0, 0)): 1.2,
    (time(18, 0, 0), time(23, 59, 59)): 0.8
}

weekend_discount = 0.7

# Function to apply discounts based on time
def apply_time_discount(row):
    start_time = row['start_datetime'].time()
    end_time = row['end_datetime'].time()
    
    # Check if it's a weekday (Monday - Friday)
    if row['start_datetime'].weekday() < 5:
        for time_range, discount in weekday_discounts.items():
            if time_range[0] <= start_time <= time_range[1] and time_range[0] <= end_time <= time_range[1]:
                return row[['moto', 'car', 'rv', 'bus', 'truck']] * discount
    else:  # Weekend (Saturday and Sunday)
        return row[['moto', 'car', 'rv', 'bus', 'truck']] * weekend_discount

# Convert id_start and id_end to datetime objects
result_with_toll_rates['start_datetime'] = pd.to_datetime(result_with_toll_rates['id_start'])
result_with_toll_rates['end_datetime'] = pd.to_datetime(result_with_toll_rates['id_end'])

# Apply time discounts
result_with_toll_rates[['moto', 'car', 'rv', 'bus', 'Truck']] = result_with_toll_rates.apply(apply_time_discount, axis=1)

# Drop unnecessary columns
result_with_toll_rates.drop(['start_datetime', 'end_datetime'], axis=1, inplace=True)

# Print the resulting DataFrame
print(result_with_toll_rates)


      id_start   id_end  distance   moto    car     rv     bus  truck  \
0      1001400  1001402       9.7  6.208  9.312  11.64  17.072  34.92   
1      1001400  1001404       0.0  0.000  0.000   0.00   0.000   0.00   
2      1001400  1001406       0.0  0.000  0.000   0.00   0.000   0.00   
3      1001400  1001408       0.0  0.000  0.000   0.00   0.000   0.00   
4      1001400  1001410       0.0  0.000  0.000   0.00   0.000   0.00   
...        ...      ...       ...    ...    ...    ...     ...    ...   
1801   1004356  1001470       0.0  0.000  0.000   0.00   0.000   0.00   
1802   1004356  1001472       0.0  0.000  0.000   0.00   0.000   0.00   
1803   1004356  1001488       4.0  2.560  3.840   4.80   7.040  14.40   
1804   1004356  1004354       2.0  1.280  1.920   2.40   3.520   7.20   
1805   1004356  1004355       0.0  0.000  0.000   0.00   0.000   0.00   

     start_day       start_time   end_day         end_time   Truck  
0     Thursday  00:00:00.001001  Thursday  00:00:00.00

In [53]:
# Reorder the columns
result_with_toll_rates = result_with_toll_rates[[
    'id_start', 'id_end', 'distance', 'start_day', 'start_time', 'end_day', 'end_time', 'moto', 'car', 'rv', 'bus', 'Truck'
]]

# Print the resulting DataFrame
print(result_with_toll_rates)


      id_start   id_end  distance start_day       start_time   end_day  \
0      1001400  1001402       9.7  Thursday  00:00:00.001001  Thursday   
1      1001400  1001404       0.0  Thursday  00:00:00.001001  Thursday   
2      1001400  1001406       0.0  Thursday  00:00:00.001001  Thursday   
3      1001400  1001408       0.0  Thursday  00:00:00.001001  Thursday   
4      1001400  1001410       0.0  Thursday  00:00:00.001001  Thursday   
...        ...      ...       ...       ...              ...       ...   
1801   1004356  1001470       0.0  Thursday  00:00:00.001004  Thursday   
1802   1004356  1001472       0.0  Thursday  00:00:00.001004  Thursday   
1803   1004356  1001488       4.0  Thursday  00:00:00.001004  Thursday   
1804   1004356  1004354       2.0  Thursday  00:00:00.001004  Thursday   
1805   1004356  1004355       0.0  Thursday  00:00:00.001004  Thursday   

             end_time   moto    car     rv     bus   Truck  
0     00:00:00.001001  6.208  9.312  11.64  17.072